In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
import torch
import scipy.sparse as sp
import glob
import warnings
import anndata
import matplotlib.pyplot as plt
import scanpy as sc
import numpy as np
import seaborn as sns
import torch
import pyro

In [4]:
import spaceoracle
from spaceoracle import SpaceOracle

In [7]:
adata_train = sc.read_h5ad('../data/flowsig/adata_train_flowsig.h5ad')
adata_train

AnnData object with n_obs × n_vars = 10277 × 4998
    obs: 'cluster', 'rctd_cluster', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts', 'spatial_kmeans'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cluster_colors', 'commot-user_database-info', 'flowsig_network', 'hvg', 'log1p', 'nsf_info', 'rctd_cluster_colors'
    obsm: 'X_flow', 'X_gem', 'X_spatial', 'commot-user_database-sum-receiver', 'commot-user_database-sum-sender', 'rctd_results', 'spatial'
    layers: 'imputed_count', 'normalized_count', 'raw_count'
    obsp: 'commot-user_database-Angptl2-Pirb', 'commot-user_database-Angptl4-Cdh5

In [22]:
# import shutil
# import os
    
# models_dir = './models'
# if os.path.exists(models_dir):
#     shutil.rmtree(models_dir)
# os.makedirs(models_dir)

In [23]:
so = spaceoracle.SpaceOracle(
    adata=adata_train,
    annot='rctd_cluster', 
    max_epochs=10, 
    learning_rate=7e-4, 
    spatial_dim=64,
    batch_size=512,
    alpha=0.1,
    test_mode=True
)

🌍️ Generating spatial 64x64 maps...


In [25]:
so.load_betadata('Prr5l', so.save_dir)

,beta0,beta_Atf3,beta_Bcl6,beta_Bclaf1,beta_Bhlhe41,beta_Brf1,beta_Cebpb,beta_Cebpd,beta_Clock,beta_Ctcf,...,H2-Q6-Cd8b1,H2-K1-Cd8b1,H2-T-ps-Cd8b1,Dll4-Notch2,Dll1-Notch2,Pecam1-Pecam1,Ptprm-Ptprm,Selplg-Selp,Sema7a-Plxnc1,rctd_cluster
AAAAACCGTCAGAC,-0.000686,-0.006857,-0.005167,-0.000210,0.012484,-0.000607,0.000680,-0.000053,0.004558,0.009868,...,0.000007,0.000090,2.102841e-07,7.747599e-08,1.651570e-09,1.068663e-06,0.0,1.353178e-07,4.006067e-07,0
AAAAACGCTCCGAC,-0.000763,-0.005183,-0.003843,-0.000203,0.016563,-0.000589,0.000650,-0.000044,0.002721,0.009339,...,0.000007,0.000091,2.614209e-07,6.337657e-08,1.245769e-13,1.062276e-06,0.0,1.187623e-07,2.637430e-07,0
AAAAAGCACGTGTT,-0.000813,-0.004162,-0.000502,0.000001,0.022498,-0.000565,0.000536,-0.000040,0.000211,0.010912,...,0.000014,0.000187,5.388192e-07,8.616721e-08,1.284353e-11,1.279367e-06,0.0,3.897501e-07,4.424786e-07,0
AAAAAGCGGCGCAC,0.009304,0.029132,-0.000517,-0.017051,0.001320,-0.012513,0.000576,-0.000271,-0.004958,-0.009332,...,0.000002,0.000050,1.764234e-07,1.994956e-08,1.036963e-17,2.420286e-07,0.0,6.539502e-08,1.167912e-07,2
AAAACCACACGCCT,-0.000694,-0.005460,-0.003810,-0.000228,0.015270,-0.000531,0.000677,-0.000044,0.003804,0.010206,...,0.000007,0.000122,3.326266e-07,6.613742e-08,2.755664e-12,8.200287e-07,0.0,1.376148e-07,2.570618e-07,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTTTTATAATACC,-0.000687,-0.006202,-0.003846,-0.000214,0.015247,-0.000456,0.000657,-0.000043,0.003546,0.010097,...,0.000011,0.000140,3.497066e-07,9.126798e-08,4.380537e-09,1.343446e-06,0.0,1.515393e-07,4.872094e-07,0
TTTTTTCGCGGCTA,-0.002960,0.026658,0.035863,0.012867,0.026109,0.025799,0.005233,0.010239,-0.009396,-0.023455,...,0.000008,0.000109,3.271232e-07,8.948369e-08,8.833382e-16,1.222870e-06,0.0,0.000000e+00,2.849969e-07,4
TTTTTTGGGCTGGT,0.009074,0.029247,-0.000558,-0.016628,0.001314,-0.012650,0.000603,-0.000262,-0.004248,-0.010019,...,0.000005,0.000120,3.618653e-07,9.176038e-08,9.596241e-21,6.111285e-07,0.0,1.683887e-07,3.165048e-07,2
TTTTTTGTACGATT,-0.000551,-0.007491,-0.007146,-0.000316,0.007182,-0.000422,0.000715,-0.000053,0.007572,0.010578,...,0.000004,0.000083,2.466721e-07,6.767524e-08,2.826720e-26,7.601525e-07,0.0,1.527822e-07,2.975485e-07,0


In [24]:
so.run()

Fitting 7 models in parallel... with 7/11 cores


/opt/homebrew/Caskroom/miniforge/base/envs/SpaceOracle/lib/python3.9/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.B

88 regulators + 108 ligand-receptor pairs


KeyboardInterrupt: 